# YOLOv8 Fine-tuning Tutorial

This notebook demonstrates how to fine-tune a YOLOv8 model using the Ultralytics library with a local dataset composed of images of men and woman. The dataset of only 39 images and a light training already shows noticable capabilities.

## Overview
- Install required libraries
- Load local dataset in YOLO format
- Configure YOLOv8 for training
- Train the model
- Evaluate and test predictions
- Export the fine-tuned model

## Prerequisites
- Python 3.8+
- CUDA-compatible GPU (recommended for faster training but not a requirement)
- Local dataset in YOLO format (images + labels)

## Introduction to YOLO (You Only Look Once)

**YOLO (You Only Look Once)** is a real-time object detection algorithm that approaches detection as a single regression problem. Unlike traditional methods that apply classifiers to different regions multiple times, YOLO processes the entire image in one forward pass, predicting bounding boxes, class probabilities simultaneously or segmenting the image.

![YOLO Architecture](https://user-images.githubusercontent.com/26833433/212094133-6bb8c21c-3d47-41df-a512-81c5931054ae.png)

The algorithm divides the input image into a grid system where each cell predicts bounding boxes and confidence scores. This single-pass architecture enables processing speeds of up to 100+ frames per second depending on the model variant and hardware, making it suitable for real-time applications.

**YOLOv8**, developed by Ultralytics, is the latest iteration in the YOLO family. It features an anchor-free design, updated data augmentation techniques, and modified loss functions. The architecture provides multiple model sizes from nano to extra-large, allowing users to select between speed and accuracy based on their requirements.

YOLO is applied across various domains including autonomous vehicles for obstacle detection, surveillance systems for monitoring, sports analytics for tracking, medical imaging for diagnosis assistance, retail automation for inventory management, and wildlife monitoring for conservation purposes.


## 1. Install Required Libraries

In [2]:
# Install Ultralytics (YOLOv8) and other dependencies
!pip install ultralytics pillow pyyaml matplotlib opencv-python -q

# Import necessary libraries
import os
from pathlib import Path
from ultralytics import YOLO
import torch
import yaml
from PIL import Image
import shutil

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.5.1+cpu
CUDA available: False



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Users\FKY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## 2. Load Local Dataset

We'll use the local "dataset HF" folder which contains:
- Images (.jpeg files)
- Labels (.txt files in YOLO format: `class_id center_x center_y width height`, all normalized to 0-1)
- classes.txt (list of class names)

Make sure your dataset folder is in the same directory as this notebook.

You can annotate your own images with many different, I used https://www.yololabellingtool.com/ to annotate this dataset, take a look a it and **try to annotate one or multiple images before continuing the notebook**. 

In [3]:
# Configuration - Using local "dataset HF" folder
DATASET_HF_PATH = Path("dataset HF")
PROJECT_DIR = Path("./yolo_finetuning")
DATASET_DIR = PROJECT_DIR / "dataset"

# Create directories
PROJECT_DIR.mkdir(exist_ok=True)
DATASET_DIR.mkdir(exist_ok=True)

print(f"Using local dataset: {DATASET_HF_PATH.absolute()}")

# Read class names from the local dataset
classes_file = DATASET_HF_PATH / "classes.txt"
if classes_file.exists():
    with open(classes_file, "r", encoding="utf-8") as f:
        class_names = [line.strip() for line in f.readlines()]
    print(f"✓ Classes found: {class_names}")
    print(f"✓ Number of classes: {len(class_names)}")
else:
    print(f"⚠️ classes.txt not found at {classes_file}")
    class_names = []

# Count files in the dataset
image_files = list(DATASET_HF_PATH.glob("*.jpeg")) + list(DATASET_HF_PATH.glob("*.jpg"))
label_files = [f for f in DATASET_HF_PATH.glob("*.txt") if f.name != "classes.txt"]

print(f"✓ Total images found: {len(image_files)}")
print(f"✓ Total label files found: {len(label_files)}")

Using local dataset: c:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Custom\dataset HF
✓ Classes found: ['homme', 'femme']
✓ Number of classes: 2
✓ Total images found: 39
✓ Total label files found: 39


## 3. Prepare Dataset in YOLO Format

YOLOv8 requires datasets in a specific format:
```
dataset/
├── train/
│   ├── images/
│   └── labels/
├── valid/
│   ├── images/
│   └── labels/
└── data.yaml
```

Labels should be in YOLO format: `class_id center_x center_y width height` (normalized 0-1)

In [4]:
import random

def prepare_yolo_dataset_from_local(source_dir, output_dir, split_ratio=0.8):
    """
    Prepare YOLO dataset from local folder
    Split into train and validation sets
    """
    # Create directory structure
    train_images_dir = output_dir / "train" / "images"
    train_labels_dir = output_dir / "train" / "labels"
    val_images_dir = output_dir / "valid" / "images"
    val_labels_dir = output_dir / "valid" / "labels"
    
    for dir_path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
        dir_path.mkdir(parents=True, exist_ok=True)
    
    # Get all image files
    image_files = list(source_dir.glob("*.jpeg")) + list(source_dir.glob("*.jpg"))
    
    # Shuffle and split
    random.seed(42)
    random.shuffle(image_files)
    split_idx = int(len(image_files) * split_ratio)
    
    train_files = image_files[:split_idx]
    val_files = image_files[split_idx:]
    
    print(f"Splitting dataset: {len(train_files)} train, {len(val_files)} validation")
    
    # Copy train files
    for img_file in train_files:
        label_file = img_file.with_suffix('.txt')
        
        # Copy image
        shutil.copy(img_file, train_images_dir / img_file.name)
        
        # Copy label if exists
        if label_file.exists():
            shutil.copy(label_file, train_labels_dir / label_file.name)
    
    # Copy validation files
    for img_file in val_files:
        label_file = img_file.with_suffix('.txt')
        
        # Copy image
        shutil.copy(img_file, val_images_dir / img_file.name)
        
        # Copy label if exists
        if label_file.exists():
            shutil.copy(label_file, val_labels_dir / label_file.name)
    
    print(f"✓ Dataset prepared successfully!")
    print(f"  Train: {len(list(train_images_dir.glob('*')))} images")
    print(f"  Valid: {len(list(val_images_dir.glob('*')))} images")
    
    return train_images_dir, val_images_dir

# Prepare the dataset
train_img_dir, val_img_dir = prepare_yolo_dataset_from_local(DATASET_HF_PATH, DATASET_DIR)

Splitting dataset: 31 train, 8 validation
✓ Dataset prepared successfully!
  Train: 31 images
  Valid: 8 images
✓ Dataset prepared successfully!
  Train: 31 images
  Valid: 8 images


In [5]:
# Verify the dataset structure
print("Dataset structure verification:")
print(f"  Train images: {DATASET_DIR / 'train' / 'images'}")
print(f"  Train labels: {DATASET_DIR / 'train' / 'labels'}")
print(f"  Valid images: {DATASET_DIR / 'valid' / 'images'}")
print(f"  Valid labels: {DATASET_DIR / 'valid' / 'labels'}")

# Count files
train_imgs = len(list((DATASET_DIR / 'train' / 'images').glob('*')))
train_lbls = len(list((DATASET_DIR / 'train' / 'labels').glob('*')))
val_imgs = len(list((DATASET_DIR / 'valid' / 'images').glob('*')))
val_lbls = len(list((DATASET_DIR / 'valid' / 'labels').glob('*')))

print(f"\n✓ Files in dataset:")
print(f"  Training: {train_imgs} images, {train_lbls} labels")
print(f"  Validation: {val_imgs} images, {val_lbls} labels")

Dataset structure verification:
  Train images: yolo_finetuning\dataset\train\images
  Train labels: yolo_finetuning\dataset\train\labels
  Valid images: yolo_finetuning\dataset\valid\images
  Valid labels: yolo_finetuning\dataset\valid\labels

✓ Files in dataset:
  Training: 31 images, 31 labels
  Validation: 8 images, 8 labels


## 4. Create data.yaml Configuration File

The `data.yaml` file tells YOLOv8 where to find the dataset and what classes exist.

In [6]:
# Define your dataset configuration using the local dataset
data_config = {
    'path': str(DATASET_DIR.absolute()),  # Root directory
    'train': 'train/images',  # Path to training images
    'val': 'valid/images',    # Path to validation images
    'nc': len(class_names),   # Number of classes from classes.txt
    'names': class_names      # Class names from classes.txt
}

# Save configuration to YAML file
yaml_path = DATASET_DIR / 'data.yaml'
with open(yaml_path, 'w', encoding='utf-8') as f:
    yaml.dump(data_config, f, default_flow_style=False, allow_unicode=True)

print(f"✓ Configuration saved to: {yaml_path}")
print("\nDataset configuration:")
print(yaml.dump(data_config, default_flow_style=False, allow_unicode=True))

✓ Configuration saved to: yolo_finetuning\dataset\data.yaml

Dataset configuration:
names:
- homme
- femme
nc: 2
path: c:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Custom\yolo_finetuning\dataset
train: train/images
val: valid/images



## 5. Load Pre-trained YOLOv8 Model

YOLOv8 comes in different sizes:
- `yolov8n.pt` - Nano (fastest, least accurate)
- `yolov8s.pt` - Small
- `yolov8m.pt` - Medium
- `yolov8l.pt` - Large
- `yolov8x.pt` - Extra Large (slowest, most accurate)

In [7]:
# Load a pre-trained YOLOv8 model
# Start with yolov8n (nano) for faster training, or use yolov8m/yolov8l for better accuracy
model = YOLO('yolov8n.pt')  # Automatically downloads if not present

print("Model loaded successfully!")
print(f"Model type: {type(model)}")

# Display model information
model.info()  # Shows model architecture details

100%|██████████| 6.25M/6.25M [00:00<00:00, 18.9MB/s]



Model loaded successfully!
Model type: <class 'ultralytics.models.yolo.model.YOLO'>
YOLOv8n summary: 225 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

## 6. Train the Model

Configure training parameters and start fine-tuning.

### Key Training Parameters:
- `data`: Path to data.yaml
- `epochs`: Number of training epochs
- `imgsz`: Image size (640 is standard)
- `batch`: Batch size (adjust based on GPU memory)
- `patience`: Early stopping patience
- `lr0`: Initial learning rate
- `device`: 'cuda' for GPU or 'cpu'

In [8]:
# Training configuration
training_args = {
    'data': str(yaml_path),      # Path to data.yaml
    'epochs': 50,                # Number of epochs (adjust based on dataset size)
    'imgsz': 640,                # Image size
    'batch': 16,                 # Batch size (reduce if GPU memory is limited)
    'patience': 10,              # Early stopping patience
    'save': True,                # Save checkpoints
    'device': 0 if torch.cuda.is_available() else 'cpu',  # Use GPU if available
    'project': str(PROJECT_DIR), # Project directory
    'name': 'yolov8_custom',     # Experiment name
    'exist_ok': True,            # Overwrite existing project
    'pretrained': True,          # Use pretrained weights
    'optimizer': 'AdamW',        # Optimizer (SGD, Adam, AdamW)
    'lr0': 0.001,                # Initial learning rate
    'weight_decay': 0.0005,      # Weight decay
    'warmup_epochs': 3,          # Warmup epochs
    'augment': True,             # Use data augmentation
    'verbose': True,             # Verbose output
}

print("Training configuration:")
for key, value in training_args.items():
    print(f"  {key}: {value}")

print("\n⚠️ Note: Training may take several hours depending on dataset size and hardware!")
print("You can monitor progress in real-time through the console output.")

Training configuration:
  data: yolo_finetuning\dataset\data.yaml
  epochs: 50
  imgsz: 640
  batch: 16
  patience: 10
  save: True
  device: cpu
  project: yolo_finetuning
  name: yolov8_custom
  exist_ok: True
  pretrained: True
  optimizer: AdamW
  lr0: 0.001
  weight_decay: 0.0005
  warmup_epochs: 3
  augment: True
  verbose: True

⚠️ Note: Training may take several hours depending on dataset size and hardware!
You can monitor progress in real-time through the console output.


In [9]:
# Start training with the local dataset
# Uncomment the following lines when you're ready to train
results = model.train(
    data=str(yaml_path),
    epochs=50,
    imgsz=640,
    batch=16,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=str(PROJECT_DIR),
    name='yolov8_custom',
    exist_ok=True
)

print("✓ Training completed!")
print(f"Results saved to: {PROJECT_DIR / 'yolov8_custom'}")

New https://pypi.org/project/ultralytics/8.3.231 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.101  Python-3.12.10 torch-2.5.1+cpu CPU (AMD Ryzen 7 PRO 7840U w/ Radeon 780M Graphics)
Ultralytics YOLOv8.2.101  Python-3.12.10 torch-2.5.1+cpu CPU (AMD Ryzen 7 PRO 7840U w/ Radeon 780M Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=yolo_finetuning\dataset\data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=yolo_finetuning, name=yolov8_custom, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=Non

train: Scanning C:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Custom\yolo_finetuning\dataset\train\labels... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<00:00, 772.41it/s]

train: New cache created: C:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Custom\yolo_finetuning\dataset\train\labels.cache



val: Scanning C:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Custom\yolo_finetuning\dataset\valid\labels... 8 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8/8 [00:00<?, ?it/s]

val: New cache created: C:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Custom\yolo_finetuning\dataset\valid\labels.cache


...

      50/50         0G     0.6202      1.136      1.085         25        640: 100%|██████████| 2/2 [00:05<00:00,  3.00s/it]
      50/50         0G     0.6202      1.136      1.085         25        640: 100%|██████████| 2/2 [00:05<00:00,  3.00s/it]/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.19it/s]

                   all          8         13      0.812      0.653      0.721      0.418



50 epochs completed in 0.098 hours.
Optimizer stripped from yolo_finetuning\yolov8_custom\weights\last.pt, 6.2MB
Optimizer stripped from yolo_finetuning\yolov8_custom\weights\last.pt, 6.2MB
Optimizer stripped from yolo_finetuning\yolov8_custom\weights\best.pt, 6.2MB

Validating yolo_finetuning\yolov8_custom\weights\best.pt...
Ultralytics YOLOv8.2.101  Python-3.12.10 torch-2.5.1+cpu CPU (AMD Ryzen 7 PRO 7840U w/ Radeon 780M Graphics)
Optimizer stripped from yolo_finetuning\yolov8_custom\weights\best.pt, 6.2MB

Validating yolo_finetuning\yolov8_custom\weights\best.pt...
Ultralytics YOLOv8.2.101  Python-3.12.10 torch-2.5.1+cpu CPU (AMD Ryzen 7 PRO 7840U w/ Radeon 780M Graphics)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.16it/s]



                   all          8         13      0.956      0.632      0.801      0.477
                 homme          7          9      0.912      0.556      0.745      0.478
                 femme          4          4          1      0.709      0.856      0.476
                 homme          7          9      0.912      0.556      0.745      0.478
                 femme          4          4          1      0.709      0.856      0.476
Speed: 1.1ms preprocess, 60.4ms inference, 0.0ms loss, 34.3ms postprocess per image
Results saved to yolo_finetuning\yolov8_custom
Speed: 1.1ms preprocess, 60.4ms inference, 0.0ms loss, 34.3ms postprocess per image
Results saved to yolo_finetuning\yolov8_custom
✓ Training completed!
Results saved to: yolo_finetuning\yolov8_custom
✓ Training completed!
Results saved to: yolo_finetuning\yolov8_custom


## 7. Evaluate the Model

After training, evaluate the model's performance on the validation set.

In [14]:
# Load the best trained model
# After training, the best model is saved as 'best.pt'
best_model_path = PROJECT_DIR / 'yolov8_custom' / 'weights' / 'best.pt'

# Check if model exists
if best_model_path.exists():
    model = YOLO(str(best_model_path))
    print(f"Loaded fine-tuned model from: {best_model_path}")
    
    # Validate the model on the validation set
    metrics = model.val(data=str(yaml_path))
    
    # Display metrics
    print("\n" + "="*50)
    print("Validation Metrics")
    print("="*50)
    print(f"mAP50: {metrics.box.map50:.4f}")        # mAP at IoU=0.50
    print(f"mAP50-95: {metrics.box.map:.4f}")       # mAP at IoU=0.50:0.95
    print(f"Precision: {metrics.box.mp:.4f}")       # Mean precision
    print(f"Recall: {metrics.box.mr:.4f}")          # Mean recall
    print("="*50)
else:
    print(f"Model not found at: {best_model_path}")
    print("Please train the model first!")
    print("\nFor demonstration, you can load the pre-trained model:")
    model = YOLO('yolov8n.pt')

Loaded fine-tuned model from: yolo_finetuning\yolov8_custom\weights\best.pt
Ultralytics YOLOv8.2.101  Python-3.12.10 torch-2.5.1+cpu CPU (AMD Ryzen 7 PRO 7840U w/ Radeon 780M Graphics)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Custom\yolo_finetuning\dataset\valid\labels.cache... 8 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8/8 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.42it/s]



                   all          8         13      0.956      0.632      0.801      0.477
                 homme          7          9      0.912      0.556      0.745      0.478
                 femme          4          4          1      0.709      0.856      0.476
                 homme          7          9      0.912      0.556      0.745      0.478
                 femme          4          4          1      0.709      0.856      0.476
Speed: 16.3ms preprocess, 41.8ms inference, 0.0ms loss, 22.5ms postprocess per image
Results saved to runs\detect\val2

Validation Metrics
mAP50: 0.8005
mAP50-95: 0.4770
Precision: 0.9561
Recall: 0.6324
Speed: 16.3ms preprocess, 41.8ms inference, 0.0ms loss, 22.5ms postprocess per image
Results saved to runs\detect\val2

Validation Metrics
mAP50: 0.8005
mAP50-95: 0.4770
Precision: 0.9561
Recall: 0.6324


## 8. Make Predictions on New Images

Test the fine-tuned model on the validation images. **Then check the result in the runs/detect/predict folders!**

In [20]:
# Make predictions on all validation images
test_image_path = DATASET_DIR / 'valid' / 'images'

# Get all validation images
val_images_list = list(test_image_path.glob('*.jpeg')) + list(test_image_path.glob('*.jpg'))

if val_images_list:
    print(f"Found {len(val_images_list)} validation images")
    print("Starting predictions on all validation images...\n")
    
    # Make predictions on all images at once
    results = model.predict(
        source=str(test_image_path),  # Pass the directory path
        conf=0.25,          # Confidence threshold
        iou=0.45,           # NMS IoU threshold
        save=True,          # Save results to disk
        show_labels=True,   # Show labels
        show_conf=True,     # Show confidence scores
        line_width=2,       # Line width of bounding boxes
    )
    
    print(f"✓ Predictions complete for {len(results)} images!")
    print(f"Results saved to runs/detect/predict")
    
    # Summary of detections
    total_detections = sum(len(r.boxes) for r in results)
    print(f"\nSummary:")
    print(f"  Total images processed: {len(results)}")
    print(f"  Total objects detected: {total_detections}")
    print(f"  Average detections per image: {total_detections/len(results):.2f}")
else:
    print("No validation images found!")

Found 8 validation images
Starting predictions on all validation images...


image 1/8 c:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Custom\yolo_finetuning\dataset\valid\images\OIP-1189158535.jpeg: 448x640 1 homme, 1 femme, 47.5ms
image 1/8 c:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Custom\yolo_finetuning\dataset\valid\images\OIP-1189158535.jpeg: 448x640 1 homme, 1 femme, 47.5ms
image 2/8 c:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Custom\yolo_finetuning\dataset\valid\images\OIP-1562483083.jpeg: 480x640 (no detections), 45.0ms
image 2/8 c:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Custom\yolo_finetuning\dataset\valid\images\OIP-1562483083.jpeg: 480x640 (no detections), 45.0ms
image 3/8 c:\Users\FKY\OneDrive - ECAM\Documents\Cours\Q1\Projet IA\2025-2026\LangChain-OpenTutorial\ECAM_Cus

## 11. Training Tips & Best Practices

### Dataset Preparation
1. **Quality over Quantity**: Ensure your annotations are accurate
2. **Balanced Classes**: Try to have roughly equal samples per class
3. **Diverse Images**: Include various lighting, angles, and backgrounds
4. **Data Augmentation**: YOLOv8 applies augmentation automatically

### Training Optimization
1. **Start Small**: Use `yolov8n` for quick experiments
2. **Monitor Training**: Watch for overfitting (validation loss increasing)
3. **Early Stopping**: Use patience parameter to stop when not improving
4. **Learning Rate**: Start with default, adjust if needed
5. **Batch Size**: Larger batch = more stable, but needs more GPU memory

### Common Issues
- **Low mAP**: More data, better annotations, longer training
- **Overfitting**: More data augmentation, reduce epochs, add regularization
- **Out of Memory**: Reduce batch size or image size
- **Slow Training**: Use GPU, reduce image size, smaller model

### Dataset Sources
- **Roboflow Universe**: Pre-labeled datasets in YOLO format
- **Hugging Face**: Various computer vision datasets
- **COCO, Pascal VOC**: Standard benchmarks
- **Custom**: Label your own with tools like LabelImg, CVAT, or Roboflow